In [1]:
import sys
import cv2
import numpy as np
import pickle
import os

# Diamond Search Algorithm

In [2]:
# Set the block size and search window size
block_size = 16
# search_window_size = 16

dx = []
dy = []

dx.append((0, 0, 0, 1, -1))
dy.append((0, 1, -1, 0, 0))

dx.append((0, -2, 0, 2, 0, -1, 1, 1, -1))
dy.append((0, 0, -2, 0, 2, 1, -1, 1, -1))

def ds(ip, pt, cf, sf, bs, lvl):
    if lvl == 0:
      return pt

    x,y = pt
    
    # Initialize the motion vector to (0, 0)
    motion_vector = pt
    
    # Initialize the minimum SSD to a large value
    min_ssd = float('inf')

    for i in range (0, 4*lvl + 1):
      x1 = x + dx[lvl - 1][i]*bs
      y1 = y + dy[lvl - 1][i]*bs
      
                
      try :
        
        # Extract the candidate block from the search frame
        block = cf[ip[1]:ip[1]+bs, ip[0]:ip[0]+bs, :]
        candidate_block = sf[y1:y1+bs, x1:x+bs, :]

        # Compute the SSD between the block and the candidate block
        ssd = np.sum((block - candidate_block)**2)
            
        # Check if the SSD is smaller than the current minimum SSD
        if ssd < min_ssd:
            # Update the motion vector and the minimum SSD
            motion_vector = (x, y)
            min_ssd = ssd
      except Exception as e:
        continue

    if motion_vector == pt:
      return ds(ip, pt, cf, sf, bs, lvl - 1)

    return ds(ip, motion_vector, cf, sf, bs, lvl)

# Compression

In [3]:
video_location = 'video.mp4'

# Load the MP4 file using OpenCV
cap = cv2.VideoCapture(video_location)

# Play video
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()

In [4]:
# Load the MP4 file using OpenCV
cap = cv2.VideoCapture(video_location)

# Get the video dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#print size of video in bytes
print ("width:", width, "  height: ", height, "  frames:", frames)
print("Size of uncompressed video in Mega bytes: ", width*height*frames*3 / (1024*1024), "MB")

width: 640   height:  360   frames: 278
Size of uncompressed video in Mega bytes:  183.251953125 MB


In [5]:
# Create a 4D NumPy array to store the video frames
video_array = np.empty((frames, height, width, 3), np.dtype('uint8'))

# Loop through the video frames and store them in the 4D array
frame_idx = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        video_array[frame_idx] = frame
        frame_idx += 1
    else:
        break

# Release the video capture object
cap.release()

In [6]:
# Initialize the compressed video array
compressed_video = []

# Loop through the frames and perform motion estimation and compensation

for i in range(0, frames - 2, 3):
    motion_vector = []

    curr_frame = video_array[i]
    compressed_video.append(curr_frame)

    try: 
        next_frame_1 = video_array[i+1]

        # Compute the motion vectors for each block using diamond search block matching
        for y in range(0, height, block_size):
            for x in range(0, width, block_size):
                motion_vector.append(ds((x,y), (x,y), curr_frame ,next_frame_1,block_size, 2))

        compressed_video.append(tuple(motion_vector))
        motion_vector.clear()
    
        
        next_frame_2 = video_array[i+2]

        # Compute the motion vectors for each block using diamond search block matching
        for y in range(0, height, block_size):
            for x in range(0, width, block_size):
                motion_vector.append(ds((x,y), (x,y), curr_frame ,next_frame_2,block_size, 2))

        compressed_video.append(tuple(motion_vector))
    except:
        continue

    print("\r", str((i+2)*100//frames) + "%", "(", i, "/", frames, ")", end='', flush=True)
print("\rDone !             ", flush=True)

Done !             


In [16]:
print(compressed_video[1][0][0])

0


# Partial Encryption

In [7]:
#function for xoring two float values

def xor_float(x, y):
    x_as_int = x.view(np.int64)
    y_as_int = y #y.view(np.int64)
    result = x_as_int ^ y_as_int
    return  result

In [8]:
# function to xor every element of a 3D array with a key
def xor_array(arr, key): 
    for i in range(len(arr)): 
        for j in range(len(arr[0])): 
            for k in range(len(arr[0][0])):
                arr[i][j][k] = xor_float(arr[i][j][k], key)

In [9]:
# import library to perform dct on rgb image
from scipy.fftpack import dct, idct

key = 1234

# Perform dct on each frame of the video
for i in range((len(compressed_video)//12)*3, len(compressed_video), 3):
    compressed_video[i] = dct(dct(compressed_video[i], axis=0, norm='ortho'), axis=1, norm='ortho')

    # Perform encryption using a key on dct coefficients  float value of each frame
    xor_array(compressed_video[i], key)

print("Key: ", key)

Key:  1234


In [10]:
# Open a file for writing
with open("compressed.mvd", 'wb') as f:
    # Use pickle to serialize and save the object to the file
    if type(compressed_video) is not None:
        pickle.dump(compressed_video, f)
    else:
        print("Error: No video to save")

print("Size of Uncompressed video  = ", width*height*frames*3 / (1024*1024), "MB")
print("size of Compressed Video    = ", os.path.getsize("compressed.mvd") / (1024*1024), "MB")
print("Compression ratio           = ", os.path.getsize("compressed.mvd")/(width*height*frames*3))

Size of Uncompressed video  =  183.251953125 MB
size of Compressed Video    =  380.14802646636963 MB
Compression ratio           =  2.0744555241223686


# Partial Decryption

### Loading From File

In [11]:
# Open the compressed MVD file for reading
with open("compressed.mvd", 'rb') as f:
    # Use pickle to load the object from the file
    compressed_video = pickle.load(f)

### Performing IDCT and Decrypting using Key

In [12]:
# Decrpyt the dct coefficients
key = 1234

for i in range((len(compressed_video)//12)*3, len(compressed_video), 3):
    xor_array(compressed_video[i], key)
    for l in range(len(compressed_video[i])): 
        for j in range(len(compressed_video[i][0])): 
            for k in range(len(compressed_video[i][0][0])):
                compressed_video[i][l][j][k] = compressed_video[i][l][j][k].view(np.float64)
    

    compressed_video[i] = idct(idct(compressed_video[i], axis=0, norm='ortho'), axis=1, norm='ortho')

# Decompression

In [13]:
frames = len(compressed_video)
decompressed_video = []

print("frames:", frames)

for i in range(0, frames - 2, 3):
    org_frame = compressed_video[i]
    decompressed_video.append(org_frame)
    try:
      motion_vector = compressed_video[i+1]
      curr_frame = np.zeros_like(org_frame)

      idx = 0

      for y in range(0, height, block_size):
          for x in range(0, width, block_size):
              x1, y1 = motion_vector[idx]
              idx = idx + 1
              curr_frame[y:y+block_size, x:x+block_size, :] = org_frame[y1:y1+block_size, x1:x1+block_size, :]

      decompressed_video.append(curr_frame)

      motion_vector = compressed_video[i+2]
      curr_frame = np.zeros_like(org_frame)
      idx = 0

      for y in range(0, height, block_size):
          for x in range(0, width, block_size):
              x1, y1 = motion_vector[idx]
              idx = idx + 1
              curr_frame[y:y+block_size, x:x+block_size, :] = org_frame[y1:y1+block_size, x1:x1+block_size, :]

      decompressed_video.append(curr_frame)
    
    except Exception as e:
      print (e)
      continue
    print("\r", str((i+2)*100//frames) + "%", "(", i, "/", frames, ")", end='', flush=True)
print("\rDone !             ", flush=True)

frames: 276
Done !             2% ( 144 / 276 )


In [14]:
# Transpose the array to change the order of dimensions
decompressed_video_t = np.moveaxis(np.array(decompressed_video), [0, 1, 2, 3], [3, 0, 1, 2])
print(decompressed_video_t.shape)

(360, 640, 3, 276)


In [15]:
# Set the frame rate of the video
fps = 30

# Create a window to display the video
cv2.namedWindow("Video")

# Iterate over the frames of the video
for i in range(decompressed_video_t.shape[3]):

    # Extract the current frame from the array
    frame = decompressed_video_t[:, :, :, i]

    # Display the frame in the window
    cv2.imshow("Video", frame)

    # Wait for a key press and exit if 'q' is pressed
    if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
        break

# Release the video window and exit
cv2.destroyAllWindows()
